okay let us just do 1 wire for now. we gonna just write the matrices. so we will be having N + 1 points in the channel ie from 0 to N. Electrons are incident only from the left and not from the right. 


We see as we apply tight binding equantions. we get H as our tridiagonal sparse matrix. with epsilons on the main diagonal. and t's hopping parameters on off diags. Now as we apply OBC (OPEN BOUNDARY CONDITIONS) we see H really is a tridiagonal sparse matrix as we had in the TBM of 1 D wire. but now we have some extra terms like t exp(ika) this goes into self energies of the contacts . contact has f1 = 1 and f2= 0. f being the fermilevel. 

we will be using the following notations

Sigma_1 , Sigma_2 for contacts , Sigma_0 for the dephasing contact. s for sources.  Gamma is anti hermitian part of Sigma. s s dagger =  Sigma (superscirpt in) / 2pi 


In [ ]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt

# Parameters
N = 100                # Number of sites (0 to N)
eps = 0.0              # Onsite energy
t = 1.0                # Hopping parameter
a = 1.0                # Lattice constant
B = 1.0                # Amplitude of incoming wave
t = 1.0      # Hopping
v = 1.0      # Electron velocity (assumed constant)
a = 1.0      # Lattice spacing
hbar = 1.0   # Planck constant (natural units)

# Use new velocity-based dispersion: hbar*v/a = -2t sin(ka)
sin_ka = -hbar * v / (2 * t * a)

if abs(sin_ka) <= 1:
    ka = np.arcsin(sin_ka)
else:
    raise ValueError("Unphysical: sin(ka) outside valid range. Choose larger t or smaller v.")


# Construct Hamiltonian (tridiagonal)
main_diag = eps * np.ones(N + 1)
off_diag = t * np.ones(N)
H = sp.diags([off_diag, main_diag, off_diag], offsets=[-1, 0, 1], format='csr')

# Construct self-energy matrices
Sigma1 = sp.lil_matrix((N+1, N+1), dtype=complex)
Sigma2 = sp.lil_matrix((N+1, N+1), dtype=complex)

Sigma1[0, 0] = t * np.exp(1j * ka)
Sigma2[N, N] = t * np.exp(1j * ka)

Sigma1 = Sigma1.tocsr()
Sigma2 = Sigma2.tocsr()

# Construct source vector s (column vector)
s = np.zeros((N + 1, 1), dtype=complex)
s[0, 0] = -2j * B * t * np.sin(ka)


# Gamma matrices: Γ = i (Σ - Σ†)
Gamma1 = 1j * (Sigma1 - Sigma1.getH())  # Sigma1† = Hermitian transpose
Gamma2 = 1j * (Sigma2 - Sigma2.getH())

# Fermilevels
f1 = 1.0
f2 = 0.0

Sig_in = f1 * Gamma1 + f2 * Gamma2

# Source vector s (already defined earlier)
# s[0, 0] = -2j * B * t * np.sin(ka)

# Compute Sig_in using s s† = Sigma_in / 2π ⇒ Sigma_in = 2π s s†
Sig_in_from_s = 2 * np.pi * (s @ s.conj().T)
print(Sig_in_from_s/(2*np.pi))

[[1.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 ...
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j ... 0.+0.j 0.+0.j 0.+0.j]]
